<a href="https://www.kaggle.com/code/nsff591/braintumordetection-conv-nn-0-998?scriptVersionId=97816295" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Convolutional Neural Network: Brain Tumor Detection**
## Importing Libraries

This dataset was taken from Ahmed Hamada in the following link: https://www.kaggle.com/datasets/ahmedhamada0/brain-tumor-detection

I rearranged the dataset a bit when I was testing some stuff offline. All credits to Ahmed Hamada for the dataset.

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

from keras.models import load_model



print("Importing Libraries Complete")

## Data Preprocessing

In [ ]:
train_data_dir= '../input/rearranged-brain-tumor-dataset-from-ahmed-hamada/DataSet'
test_data_dir= '../input/rearranged-brain-tumor-dataset-from-ahmed-hamada/TestDataSet'
model_save_dir= './'

batch_size = 32
img_width = 180
img_height = 180
val_split = 0.2
seed = 753

number_of_filters = 32
number_of_epochs = 7
number_of_units = 300

### Preprocessing the training set 

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        validation_split=0.2)

training_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

### Preprocessing the validation set

In [ ]:
validation_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

# Based on example code of keras data preprocessing api doc and Salik Hussaini example code#
validation_set = validation_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

### TensorFlow Prefetching Optimization

In [ ]:
# This does not currently work 
# AUTOTUNE = tf.data.AUTOTUNE

# training_set = training_set.cache().prefetch(buffer_size=AUTOTUNE)
# test_set = test_set.cache().prefetch(buffer_size=AUTOTUNE)

## Building the Convolutional Neural Network: Model

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=number_of_filters,kernel_size=3,activation='relu',input_shape=[img_width,img_height,3]))

### Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Adding a Second Convolution Layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=number_of_filters,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

### Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=number_of_units,activation='relu'))

### Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Training The Convolutional Neural Network

### Compiling the CNN

In [ ]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
fitted_model= cnn.fit(x=training_set,validation_data=validation_set,epochs=number_of_epochs)

## Model Summary

In [ ]:
tf.keras.utils.plot_model(cnn,  show_shapes=True,expand_nested=True)

## Evaluating the model on test data



In [ ]:
test_labels = listdir(test_data_dir)
test_image_array= []
number_of_files= 0.
prediction_accuracy= 0

for folder_number in range(len(test_labels)):
  for path in listdir(test_data_dir+'/'+test_labels[folder_number]):
    number_of_files+=1

    test_image= image.load_img(path=test_data_dir+'/'+test_labels[folder_number]+'/'+path,target_size=(img_width,img_height))
    test_image= image.img_to_array(test_image)

    normalized_test_image = test_image*1./255

    test_image= np.expand_dims(normalized_test_image,axis=0)
    prediction= cnn.predict(test_image)
    #changing the prediction "numbers" to classnames
    if prediction >= 0.5:
      prediction= 'yes'
    else:
      prediction= 'no'
    if prediction == test_labels[folder_number]:
      prediction_accuracy+=1

prediction_accuracy= prediction_accuracy/number_of_files

print('prediction accuracy= '+ str(prediction_accuracy*100)+'%')

## Accuracy Plot

In [ ]:
def plot_metrics(history):
  metrics = ['loss', 'accuracy']
  for n, metric in enumerate(metrics):
    try:
      name = metric.replace("_"," ").capitalize()
      plt.plot(history.epoch, history.history[metric], label='Train')
      plt.plot(history.epoch, history.history['val_'+metric], linestyle="--", label='Val')
      plt.xlabel('Epoch')
      plt.ylabel(name)
      if metric == 'loss':
        plt.ylim([0, plt.ylim()[1]])
      elif metric == 'auc':
        plt.ylim([0.8,1])
      else:
        plt.ylim([0,1])
      plt.legend()
      plt.show()  
    except:
      pass
plot_metrics(fitted_model)

plt.title(label='Zoomed in Accuracy plot')
plt.plot(fitted_model.history["accuracy"],label='Train')
plt.plot(fitted_model.history["val_accuracy"],linestyle="--",label='Validation')
plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

## Saving the model

In [ ]:
fitted_model.model.save(model_save_dir)
print("Saved model to disk")

## Loading the model

In [ ]:
model = tf.keras.models.load_model(model_save_dir)
model.summary()